In [1]:
import openai
import tiktoken
import pinecone
import os
from pymongo.mongo_client import MongoClient
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")

PINECONE_API_KEY = 'YOUR-PINECONE-KEY'
PINECONE_API_ENV = 'YOUR-PINECONE-ENV'
MONGO_URI = os.getenv("MONGO_URI")

/Users/mayt/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
text = '''#［肥料自給　これで強化］（１）　堆肥入りペレット　まきやすく広域流通も

埼玉県加須市で水稲など５７ヘクタールを栽培する小倉祐一さん（４２）は３月、肥料成分や有機物を水田に供給するペレット状の資材を初めてまいた。資材の原料の約７割は牛ふん堆肥を中心とした国内の資源だ。小倉さんは「ブロードキャスターが使えて省力的。臭いも気にならない」という。
農水省によると、２月の肥料価格は２０２０年を１００とした指数で１５５で、前年同月を４割上回る。輸入に大きく依存する肥料原料のリン安や塩化カリの高騰が主因。こうした中で国内資源の家畜ふん堆肥への注目が高まっている。
小倉さんはリンやカリの成分量を抑えた安価なＬ型肥料への切り替えで、肥料費の圧縮を目指す。一方、減らした分のリン、カリに加え、ケイ酸、地力向上につながる有機物の補給へ、採用したペレット資材が「稲サポ」だ。１袋（１５キロ）当たりの肥料成分比はリン酸７％、カリ６％、ケイ酸８％などで、１０アール当たり５０キロを計４ヘクタールに散布した。
同省によると、家畜排せつ物は年間約８０００万トン（２０２０年）発生し、８６％が主に堆肥として活用されている。一方、稲作地帯は畜産農家が少ないなどで水田への堆肥の施用量は１５年までの３０年間で４分の１に減った。
小倉さんも、牛ふん堆肥の施用を考えたが、近隣に入手可能な農場がなかったことや散布機を持っていないこと、臭いへの懸念などから諦めてきた。稲サポのように、ペレット加工して体積を減らした資材は、堆肥では難しかった広域での流通や、従来の散布機の活用も可能になる。
稲サポを製造する朝日アグリアは、牛ふん堆肥を群馬や埼玉県の畜産農家から仕入れる。２１年に開発して以降、埼玉や茨城、千葉といった関東圏に加え、新潟などでも利用が広がる。
・地力向上し収量安定に
家畜ふん堆肥の利用拡大へ、農研機構九州沖縄農業研究センターの荒川祐介氏は「効果や価値を改めて理解する必要がある」と指摘する。ペレット資材も含め、堆肥は家畜ふんや植物の繊維などの有機物が供給できる。短期的には収益に結び付きにくいが、長期的には、有機物を微生物が分解して土中に細かな土壌の粒ができることで、排水性と保水性が向上する。収量や品質の安定につながるとする。（川崎勇）
◇
肥料価格の高騰が続いている。国内にある資源を肥料に生かす取り組みについて、普及、定着の課題を探る。


写真説明：総合１＝肥料自給　これで強化・①家畜ふん堆肥・ペレットでまきやすく・地力向上に期待・付加価値の理解必要／〇日農／営農生活部／川崎勇／堆肥入りの肥料で地力の向上に期待する小倉さん（埼玉県加須市で）


'''


In [2]:
def nouns_extraction(text, model="gpt-3.5-turbo"):
    system_message_prompt = """Your job is to pick out all proper nouns, people's names, places' name, brand names, and quoted names in Japanese for your peer translator, and also include all words where you are not confident the English translation. You will attempt to translate the words.

For companies, use the romanji name. For example, "明治生命" => "Meiji Life Insurance Co.".

You will respond in the following format:

Japanese|English|Abbreviation|Romanji|Type
農泊|Nouhaku (Agritourism)||nouhaku|Proper Noun
農水省|Ministry of Agriculture, Forestry and Fisheries|(MAFF)|nousuishou|Government Department
幸南食糧|Kounan Shokuryo Co.||kounan shokuryo|Company
鶏がゆ|chicken porridge||tori kayu|Food

"""
    message = [
        {"role": "system", "content": system_message_prompt},
        {"role": "user", "content": text},
    ]
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=message,
                temperature=0.6,
                max_tokens=1000,
            )

            return response.choices[0].message.content
        except Exception as e:
            print(e)
    raise Exception("Failed to get response from OpenAI")

#dictionary = nouns_extraction(text, "gpt-4")
#print(dictionary)


In [65]:
from supabase import create_client, Client
from uuid import UUID
article_id = "28292167-fcc9-4edf-acbb-5e96c7b1af68"

client = MongoClient(MONGO_URI, uuidRepresentation='standard')

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)
for line in dictionary.strip().split('\n'):
    line = line.split("|")
    words = {"jp":line[0],"en": line[1],"abbreviation": line[2],"romanji": line[3],"type": line[4]}
    client.agrinews.get_collection("articles").update_one({"id": UUID(article_id)}, {"$push": {"dictionary": words}})
    client.agrinews.get_collection("dictionary").insert_one(words)

    print(dictionary)


埼玉県加須市|Saitama Prefecture, Kazo City||saitama ken kazo shi|Place
小倉祐一|Yuichi Ogura||ogura yuichi|Person
農水省|Ministry of Agriculture, Forestry and Fisheries|(MAFF)|nousuishou|Government Department
リン安|Phosphorus||rin-an|Chemical
塩化カリ|Potassium Chloride||enkakari|Chemical
Ｌ型肥料|L-type fertilizer||L-gata hiryou|Fertilizer
稲サポ|Ina-sapo||ina-sapo|Proper Noun
朝日アグリア|Asahi Agria Co.||asahi aguria|Company
牛ふん堆肥|Cow manure compost||ushi fun taihi|Material
群馬|Gunma Prefecture||gunma|Place
茨城|Ibaraki Prefecture||ibaraki|Place
千葉|Chiba Prefecture||chiba|Place
新潟|Niigata Prefecture||niigata|Place
農研機構九州沖縄農業研究センター|Kyushu Okinawa Agricultural Research Center of the National Agriculture and Food Research Organization||noukenkikou kyushu okinawa nougyou kenkyu center|Research Center
荒川祐介|Yusuke Arakawa||arakawa yusuke|Person
川崎勇|Isamu Kawasaki||kawasaki isamu|Person
埼玉県加須市|Saitama Prefecture, Kazo City||saitama ken kazo shi|Place
小倉祐一|Yuichi Ogura||ogura yuichi|Person
農水省|Ministry of Agriculture, Forest

In [8]:
client = MongoClient(MONGO_URI, uuidRepresentation='standard')
all_articles = client.agrinews.get_collection("articles").find({"dictionary": {"$exists": False}})


In [9]:
all_articles[0]


IndexError: no such item for Cursor instance

In [5]:
for (i, article) in enumerate(all_articles):
    print("Processing", i, article["id"], article["見出し"], article["本文文字数"])
    dictionary = nouns_extraction(article["text"], "gpt-4")
    for line in dictionary.strip().split('\n'):
        line = line.split("|")
        try:
            words = {"jp":line[0],"en": line[1],"abbreviation": line[2],"romanji": line[3],"type": line[4]}
            client.agrinews.get_collection("articles").update_one({"id": article["id"]}, {"$push": {"dictionary": words}})
            client.agrinews.get_collection("dictionary").insert_one({"article_id": article["id"], "article_title": article["見出し"], **words})
        except Exception as e:
            print(e)
            raise e
    print("Finished", i, article["id"], article["見出し"])

Processing 0 39c5bc81-26e7-4b4b-8250-35c41f8192ba Ｇ７警備徹底を　警察庁長官 170
Finished 0 39c5bc81-26e7-4b4b-8250-35c41f8192ba Ｇ７警備徹底を　警察庁長官
Processing 1 9ff9bfce-efc7-422d-b8e2-4c25aa562838 ［女の階段］　教わった行事食　次代に 424
Finished 1 9ff9bfce-efc7-422d-b8e2-4c25aa562838 ［女の階段］　教わった行事食　次代に
Processing 2 01022830-c289-4980-af24-d0e6ad203f0e ［週間天気予報］ 8
Finished 2 01022830-c289-4980-af24-d0e6ad203f0e ［週間天気予報］
